In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,27160
2,Huelva,Confirmados PDIA 14 días,1454
3,Huelva,Tasa PDIA 14 días,"283,3369058986301"
4,Huelva,Confirmados PDIA 7 días,611
5,Huelva,Total Confirmados,27334
6,Huelva,Curados,22801
7,Huelva,Fallecidos,356


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  27160.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7048.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1219 personas en los últimos 7 días 

Un positivo PCR cada 462 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,27160.0,611.0,1454.0,513170.0,119.063858,283.336906,283.0
Condado-Campiña,8253.0,306.0,737.0,156231.0,195.863817,471.737363,135.0
Huelva-Costa,16058.0,249.0,599.0,289548.0,85.996104,206.874162,121.0
Almonte,1295.0,127.0,254.0,24507.0,518.219284,1036.438569,62.0
Huelva (capital),7048.0,118.0,311.0,143837.0,82.037306,216.216968,57.0
Cartaya,1189.0,82.0,173.0,20083.0,408.305532,861.425086,42.0
Sierra de Huelva-Andévalo Central,2419.0,47.0,88.0,67391.0,69.742250,130.581235,22.0
Moguer,999.0,42.0,104.0,21867.0,192.070243,475.602506,19.0
Rociana del Condado,451.0,31.0,70.0,7939.0,390.477390,881.723139,16.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Almonte,1295.0,127.0,254.0,24507.0,518.219284,1036.438569,62.0
San Bartolomé de la Torre,275.0,9.0,38.0,3761.0,239.298059,1010.369583,1.0
Rociana del Condado,451.0,31.0,70.0,7939.0,390.477390,881.723139,16.0
Cartaya,1189.0,82.0,173.0,20083.0,408.305532,861.425086,42.0
Lucena del Puerto,185.0,9.0,22.0,3261.0,275.988960,674.639681,2.0
Palos de la Frontera,548.0,29.0,59.0,11742.0,246.976665,502.469767,7.0
Moguer,999.0,42.0,104.0,21867.0,192.070243,475.602506,19.0
Condado-Campiña,8253.0,306.0,737.0,156231.0,195.863817,471.737363,135.0
Bonares,223.0,4.0,28.0,6060.0,66.006601,462.046205,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Gibraleón,556.0,1.0,10.0,12737.0,7.851142,78.511423,0.0,0.100000
Villalba del Alcor,470.0,1.0,8.0,3366.0,29.708853,237.670826,0.0,0.125000
Bonares,223.0,4.0,28.0,6060.0,66.006601,462.046205,4.0,0.142857
San Juan del Puerto,408.0,5.0,33.0,9411.0,53.129317,350.653491,1.0,0.151515
San Bartolomé de la Torre,275.0,9.0,38.0,3761.0,239.298059,1010.369583,1.0,0.236842
Villarrasa,141.0,2.0,8.0,2211.0,90.456807,361.827227,2.0,0.250000
Punta Umbría,683.0,4.0,15.0,15355.0,26.050147,97.688049,0.0,0.266667
Aljaraque,971.0,11.0,37.0,21474.0,51.224737,172.301388,2.0,0.297297
Palma del Condado (La),1012.0,5.0,16.0,10801.0,46.292010,148.134432,1.0,0.312500
